### Notes

The models you can run here will be limited by what's implemented by the litellm library, because that's what managing the API calls.

You can see the models here: https://docs.litellm.ai/docs/



In [1]:
from functions import *

## Parameters

In [2]:
num_perturbations = 5
models_dict = {'claude-2.1':  "ANTHROPIC", 'gpt-3.5-turbo-0301': "OPENAI"
               } 
csv_file_path = '../data/prompt_target_answer_pairs.csv'
similarity_model_name = 'sentence-transformers/paraphrase-mpnet-base-v2'
num_runs = 2
variable_temperature = .9



In [4]:
prompts = [
    {"prompt": "What is the capital of France?"},
    {"prompt": "What is the boiling point of water?"}
]
instructions = "Please answer thoroughly: "
results = process_prompts(prompts, models_dict, num_perturbations=5, num_runs=1, is_file_path=True, generate_perturbations=True, perform_similarity=False, instructions ="Please answer as briefly as possible: ", variable_temperature=False
)


2023-12-01 11:35:52,564 - INFO - Using direct list of prompts.
2023-12-01 11:36:03,878 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-01 11:36:06,318 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-01 11:36:06,318 - INFO - Processing complete.


In [5]:
results

,model,original_prompt,response,temperature
0,claude-2.1,What is the capital of France?,The capital of France is Paris.,0.259931
1,claude-2.1,What is the boiling point of water?,The boiling point of water is 100 degrees Cel...,0.038786
2,gpt-3.5-turbo-0301,What is the capital of France?,The capital of France is Paris. It is located ...,0.050149
3,gpt-3.5-turbo-0301,What is the boiling point of water?,The boiling point of pure water at standard at...,0.915284


## Responses from LLMs

### With Similarity Scores

In [7]:
results ['similarity_score'] = calculate_similarity(results ['target_answer'].tolist(), results['response'].tolist(), similarity_model_name)
results


KeyError: 'target_answer'

### Best and Worst Response from Each Model/Prompt Combo

In [5]:
aggregate_results(responses_df)

,model,original_prompt,best_answer,best_similarity,worst_answer,worst_similarity
0,gpt-3.5-turbo-0301,What is the powerhouse of the cell?,The mitochondria is known as the energy factor...,0.839692,The primary source of energy production in the...,0.537239
1,gpt-3.5-turbo-0301,What is the basic unit of life?,The fundamental entity that makes up all livin...,0.834349,The smallest structural and functional unit of...,0.712358
2,gpt-3.5-turbo-0301,What do genes consist of?,Genes are composed of DNA (deoxyribonucleic ac...,0.815171,Genetic structures consist of DNA (deoxyribonu...,0.681154
3,gpt-3.5-turbo-0301,How do astroflora perform photosynthesis in lo...,"Astroflora, or plants that grow in space, have...",0.806800,"As an AI language model, I do not have informa...",0.703187
4,claude-2.1,What is the powerhouse of the cell?,The mitochondria are the intracellular struct...,0.804405,Aerobic respiration and most ATP synthesis ta...,0.486933
5,claude-2.1,What do genes consist of?,Genes are made up of DNA (deoxyribonucleic ac...,0.779261,The basic building blocks of genes are nucleo...,0.650256
6,gpt-3.5-turbo-0301,What particle stabilizes the membrane of hover...,"As an AI language model, I cannot find any spe...",0.756064,The particle is called dermaseptin.,0.332667
7,claude-2.1,What is the basic unit of life?,The most basic component that makes up all li...,0.754846,"There is no definitive ""simplest form of matt...",0.488756
8,claude-2.1,What particle stabilizes the membrane of hover...,"Unfortunately, I do not have enough context t...",0.714272,"Unfortunately, I do not have enough context t...",0.562627
9,claude-2.1,How do astroflora perform photosynthesis in lo...,"Unfortunately, I do not have enough context t...",0.632290,There are no known plants native to outer spa...,0.385661
